In [1]:
import os
import cv2
import numpy as np
from config import *
from utils import interpolate_image, DataGenerator, create_directory
import segmentation_models as sm
import tensorflow as tf
from tensorflow.keras import callbacks

Segmentation Models: using `keras` framework.


In [2]:
model_path = create_directory(["models"])
training_path = "DATA/compressed"
training_images = os.listdir(training_path)
training_images = [f"{training_path}/{e}" for e in training_images]

models already existed


In [3]:
train_data = DataGenerator(np.array(training_images),4)

In [4]:
sm.set_framework('tf.keras')
sm.framework()

model = sm.Unet(BACKBONE, classes=n_classes, activation=activation,
                input_shape=(input_images_size, input_images_size, 1),
                encoder_weights=None)
optim = tf.keras.optimizers.Adam(learning_rate)

dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5),
           sm.metrics.FScore(threshold=0.5)]

model.compile(optim, total_loss, metrics)

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 40) 360         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 40) 160         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 40) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

Total params: 17,867,113
Trainable params: 17,777,833
Non-trainable params: 89,280
__________________________________________________________________________________________________


In [6]:
tensorboard_callback = callbacks.TensorBoard(log_dir="tensor.logs")
best = {"val":-999}
class SaveModelCallBack(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        self.model.save_weights("models/model.hdf5")
        iou_score = logs.get("iou_score")
        if iou_score > best["val"]:
            best['val'] = iou_score
            model.save_weights("models/weights.hdf5")
            model.save("models/best_model.hdf5")

save_models_callback = SaveModelCallBack()
results = model.fit(train_data,
                    initial_epoch=0,
                    epochs=epochs,
                    callbacks=[tensorboard_callback,
                               save_models_callback])


/home/hassanrehman/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/hassanrehman/anaconda3/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/100
165/165 [==============================] - 107s 507ms/step - loss: 0.4749 - iou_score: 0.5297 - f1-score: 0.6694
Epoch 2/100
165/165 [==============================] - 81s 490ms/step - loss: 0.1580 - iou_score: 0.7787 - f1-score: 0.8746
Epoch 3/100
165/165 [==============================] - 82s 492ms/step - loss: 0.1163 - iou_score: 0.8306 - f1-score: 0.9070
Epoch 4/100
165/165 [==============================] - 81s 490ms/step - loss: 0.1025 - iou_score: 0.8491 - f1-score: 0.9180
Epoch 5/100
165/165 [==============================] - 82s 491ms/step - loss: 0.0948 - iou_score: 0.8592 - f1-score: 0.9240
Epoch 6/100
165/165 [==============================] - 81s 490ms/step - loss: 0.0796 - iou_score: 0.8799 - f1-score: 0.9359
Epoch 7/100
165/165 [==============================] - 81s 491ms/step - loss: 0.0752 - iou_score: 0.8859 - f1-score: 0.9394
Epoch 8/100
165/165 [==============================] - 81s 490ms/step - loss: 0.0796 - iou_score: 0.8800 - f1-score: 0.9358
Epoch 9